In [1]:
import numpy as np
from scipy import stats

### 🔹 [두 모집단 평균 차이 검정: 정규성 X]
**문제:**

한 카페는 **BGM(배경음악)** 유무가 손님의 **체류시간**에 영향을 주는지 알고 싶다.  
데이터를 수집했더니 아래와 같았고, **정규성은 만족하지 않는다고 한다.**

| 그룹       | 체류시간(분)                                       |
|------------|-----------------------------------------------------|
| BGM 있음   | 42, 43, 45, 45, 46, 46, 47, 48, 48, 49, 50, 52, 55 |
| BGM 없음   | 31, 32, 32, 33, 33, 34, 35, 36, 37, 37, 38, 40, 60 |

→ 두 그룹의 체류시간에 **유의미한 차이가 있는가?** (유의수준 0.05)

In [4]:
group_a = [42, 43, 45, 45, 46, 46, 47, 48, 48, 49, 50, 52, 55]
group_b = [31, 32, 32, 33, 33, 34, 35, 36, 37, 37, 38, 40, 60]

In [5]:
# 01 정규성 검정
a_statsitic, a_p_value = stats.shapiro(group_a)
b_statsitic, b_p_value = stats.shapiro(group_b)
_H0 = '정규성을 만족한다.'
_H1 = '정규성을 만족하지 않는다.'


print(f'A p_value : {a_p_value}')
print(f'B p_value : {b_p_value}')
if 0.05 < a_p_value and 0.05 < b_p_value:
    print(_H0)
else:
    print(_H1)

A p_value : 0.8746099455541976
B p_value : 0.00018405580657091574
정규성을 만족하지 않는다.


In [6]:
# 정규성 가정을 하지 않아서, 데이터 분포가 왜곡돼도 사용할 수 있게 중앙값을 본다고 한다.
# 정확히는 중앙값이 아닌 순위 기반 계산
H0 = "두 집단의 중앙값은 같다 (차이 없다)"
H1 = "두 집단의 중앙값은 다르다 (차이 있다)"

# Mann-Whitney U 검정
statistic, p_value = stats.mannwhitneyu(group_a, group_b, alternative='two-sided')


print(f'U 통계량: {statistic}')
print(f'p-value: {p_value}')
if p_value < 0.05:
    print(H1)
else:
    print(H0)

U 통계량: 156.0
p-value: 0.0002676354274025252
두 집단의 중앙값은 다르다 (차이 있다)


### 🔹 [두 모집단 평균 차이 검정: 정규성 O, 등분산성 X]

한 기업은 **직원 교육 프로그램의 종류에 따라 문제 해결 능력**이 달라지는지를 알고 싶다.  
직원들을 무작위로 나누어 A 프로그램과 B 프로그램을 이수하게 하고,  
이후 **문제 해결 테스트 점수(100점 만점)**를 기록하였다.

아래는 두 집단의 점수이다:

| 교육 프로그램 | 점수 |
|----------------|------|
| A 프로그램 | 85, 86, 87, 87, 88, 89, 90, 90 |
| B 프로그램 | 60, 70, 75, 80, 85, 90, 95, 100 |

- 정규성 검정 결과: **두 집단 모두 정규분포를 따름**
- 등분산성 검정 결과: **두 집단의 분산이 다름 (등분산X)**  
- 유의수준: 0.05

In [7]:
group_a = [85, 86, 87, 87, 88, 89, 90, 90]
group_b = [60, 70, 75, 80, 85, 90, 95, 100]

H0 = '두 프로그램 간 문제 해결 능력차이가 없다.'
H1 = '두 프로그램 간 문제 해결 능력차이가 있다.'

In [8]:
# 01 정규성 검정

a_statsitic, a_p_value = stats.shapiro(group_a)
b_statsitic, b_p_value = stats.shapiro(group_b)
_H0 = '정규성을 만족한다.'
_H1 = '정규성을 만족하지 않는다.'


print(f'A p_value : {a_p_value}')
print(f'B p_value : {b_p_value}')
if 0.05 < a_p_value and 0.05 < b_p_value:
    print(_H0)
else:
    print(_H1)

A p_value : 0.6001739116043654
B p_value : 0.9782133905594868
정규성을 만족한다.


In [9]:
# 02 등분산성 검정

statsitic, p_value = stats.levene(group_a, group_b)
_H0 = '등분산성을 만족한다.'
_H1 = '등분산성을 만족하지 않는다.'

print(f'p_value : {p_value}')
if 0.05 < p_value:
    print(_H0)
else:
    print(_H1)

p_value : 0.0027035475005920606
등분산성을 만족하지 않는다.


In [10]:
# 03 Welch's t-test
# t-test 에서 equal_var = False
statsitic, p_value = stats.ttest_ind(group_a, group_b, equal_var=False)

print(f'p_value : {p_value}')
if 0.05 < p_value:
    print(H0)
else:
    print(H1)

p_value : 0.2558121926532246
두 프로그램 간 문제 해결 능력차이가 없다.


In [11]:
# 신뢰구간 검정
# ## 평균과 분산
mean_a, mean_b = np.mean(group_a), np.mean(group_b)
var_a, var_b = np.var(group_a, ddof=1), np.var(group_b, ddof=1)
n_a, n_b = len(group_a), len(group_b)

# ## 표준오차
se = np.sqrt(var_a / n_a + var_b / n_b)

# ## 자유도 (Welch 방식)
dof = (var_a / n_a + var_b / n_b)**2 / ((var_a**2) / ((n_a**2) * (n_a - 1)) + (var_b**2) / ((n_b**2) * (n_b - 1)))

# ## 임계값
alpha = 0.05
t_critical = stats.t.ppf(1 - alpha / 2, df=dof)

# ## 신뢰구간
mean_diff = mean_a - mean_b
ci_lower = mean_diff - t_critical * se
ci_upper = mean_diff + t_critical * se

print(f"Welch's t-test 신뢰구간: ({ci_lower:.4f}, {ci_upper:.4f})")

if ci_lower < 0 < ci_upper:
    print("→ 귀무가설 채택 (평균 차이 없음)")
else:
    print("→ 대립가설 채택 (평균 차이 있음)")

Welch's t-test 신뢰구간: (-5.3051, 17.0551)
→ 귀무가설 채택 (평균 차이 없음)


### 🔹 [두 모집단 평균 차이 검정: 정규성 O, 등분산성 O]

**문제:**

한 스타트업은 **재택근무**와 **출근근무** 방식이  
**직원들의 집중력**에 영향을 미치는지를 알고 싶다.  

직원들을 두 집단으로 나눠 **업무 집중력 점수**를 측정하였다 (100점 만점).

| 그룹 | 점수 |
|------|------|
| 재택근무 | 82, 85, 88, 90, 86, 84, 87 |
| 출근근무 | 78, 80, 79, 75, 77, 76, 78 |

- 유의수준: 0.05  
- 두 집단은 서로 **독립된 집단**  
- → **재택근무와 출근근무 간 집중력 평균에 유의미한 차이가 있는가?**

In [12]:
group_a = [82, 85, 88, 90, 86, 84, 87]
group_b = [78, 80, 79, 75, 77, 76, 78]

H0 = '재택근무와 출근근무 간 집중력 평균에 유의미한 차이가 없다.'
H1 = '재택근무와 출근근무 간 집중력 평균에 유의미한 차이가 있다.'

In [13]:
# 01 정규성 검정

a_statsitic, a_p_value = stats.shapiro(group_a)
b_statsitic, b_p_value = stats.shapiro(group_b)
_H0 = '정규성을 만족한다.'
_H1 = '정규성을 만족하지 않는다.'


print(f'A p_value : {a_p_value}')
print(f'B p_value : {b_p_value}')
if 0.05 < a_p_value and 0.05 < b_p_value:
    print(_H0)
else:
    print(_H1)

A p_value : 0.9999993334145784
B p_value : 0.957665102423992
정규성을 만족한다.


In [14]:
# 02 등분산성 검정

statsitic, p_value = stats.levene(group_a, group_b)
_H0 = '등분산성을 만족한다.'
_H1 = '등분산성을 만족하지 않는다.'

print(f'p_value : {p_value}')
if 0.05 < p_value:
    print(_H0)
else:
    print(_H1)

p_value : 0.3370490579535843
등분산성을 만족한다.


In [15]:
# 03 t-검정
statistic, p_value = stats.ttest_ind(group_a, group_b, alternative='two-sided')
statistic, p_value

(7.068699114903216, 1.3031725301697406e-05)

In [16]:
# 통계량으로 해석
alpha = 0.05
dof = len(group_a) + len(group_b) - 2
critical = stats.t.ppf(1 - alpha / 2, df=dof)

print(f'statistic   : {statistic}')
print(f'critical   : {critical}')
if abs(statistic) < critical:
    print(H0)
else:
    print(H1)

statistic   : 7.068699114903216
critical   : 2.1788128296634177
재택근무와 출근근무 간 집중력 평균에 유의미한 차이가 있다.


In [17]:
# p_value로 해석
alpha = 0.05
dof = len(group_a) + len(group_b) - 2

p_value = 2 * (1 - stats.t.cdf(abs(statistic), df=dof))
p_value

print(f'p_value   : {p_value}')
if 0.05 < p_value:
    print(H0)
else:
    print(H1)

p_value   : 1.3031725301759778e-05
재택근무와 출근근무 간 집중력 평균에 유의미한 차이가 있다.


In [18]:
# 신뢰구간 검정
mean_a = np.mean(group_a)
var_a = np.var(group_a, ddof=1)
n_a = len(group_a)

mean_b = np.mean(group_b)
var_b = np.var(group_b, ddof=1)
n_b = len(group_b)

# 자유도
dof = n_a + n_b - 2

# 표준오차
standard_error = np.sqrt((var_a / n_a) + (var_b / n_b))

# 신뢰도
alpha = 0.05
t_critical = stats.t.ppf(1 - alpha / 2, df=dof)

# 신뢰구간
mean_diff = mean_a - mean_b
ci_lower = mean_diff - (t_critical * standard_error)
ci_upper = mean_diff + (t_critical * standard_error)

print(f"신뢰구간 {ci_lower} ~ {ci_upper}")
if ci_lower < 0 < ci_upper:
    print(H0)
else:
    print(H1)

신뢰구간 5.8305998264717145 ~ 11.026543030671146
재택근무와 출근근무 간 집중력 평균에 유의미한 차이가 있다.
